# FFF Step 2: Scaffold selection

## Pre-requisites

- [ ] BulkDock & HIPPO set up for Target
- [ ] Algorithmic merges generated and placed
- [ ] Merges uploaded to Fragalysis and curated

In [4]:
%load_ext autoreload
%autoreload 2
import hippo
from os import environ, system
from pathlib import Path
import json
from mrich import print
import mrich
import shutil
import molparse as mp

In [5]:
target_name = "D68EV3C"

In [6]:
animal = hippo.HIPPO(target_name, Path(environ["BULK"])/"TARGETS"/target_name/f"{target_name}.sqlite")

 Creating HIPPO animal

name = D68EV3C

db_path = /opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/D68EV3C/D68EV3C.sqlite

DEBUG: hippo.Database.__init__()

DEBUG: Database.path = /opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/D68EV3C/D68EV3C.sqlite

DEBUG: hippo.Database.connect()

DEBUG: sqlite3.version='2.6.0'

 Success  Database connected @ /opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/D68EV3C/D68EV3C.sqlite!

 Success  Initialised animal HIPPO("D68EV3C")!

## Add Manual designs

In [1]:
manual_designs = [
    # populate tuples in this format:
    # (smiles, alias, inspirations),
]

In [2]:
for smiles, alias, inspirations in manual_designs:

    print(smiles, alias, inspirations)

# Export SMILES for Chem Review

In [7]:
animal.tags.unique

{'BulkDock Fragalysis export',
 'Fragmenstein placed',
 '[Other] iter1_frags',
 '[Other] upload_1 2025-02-11',
 'hits'}

In [9]:
animal.poses(tag="BulkDock Fragalysis export").compounds.write_smiles_csv("")

{C × 148}